In [1]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from sklearn.model_selection import train_test_split

### Load the dataset and pre-process them into tensors

In [2]:
data_set = pd.read_csv("data/cipher_objective.csv")
df_x = data_set["text"]
df_y = data_set["class"]

arr_x = []
arr_y = []

for i in range(len(df_x)):
    text = df_x.iloc[i]
    temp = [ord(c) for c in text]
    arr_x.append(temp)

    label = df_y.iloc[i]
    arr_y.append(0 if label == "exit" else 1)

data_x = torch.tensor(arr_x, dtype=torch.float32)
data_y = torch.tensor(arr_y, dtype=torch.long)

### Split the dataset and load them into data loaders

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
  data_x, data_y,
  test_size=0.2,
  stratify=data_y,
  random_state=42
)

train_x = X_train.detach().clone()
train_y = y_train.detach().clone()
test_x = X_test.detach().clone()
test_y = y_test.detach().clone()

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

torch.Size([3200, 100]) torch.Size([3200]) torch.Size([800, 100]) torch.Size([800])


In [4]:
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1600, shuffle=True)

### Training part

In [5]:
class Model(nn.Module):
  def __init__(self, input_size: int):
    super().__init__()
    self.l1 = nn.Linear(input_size, input_size, True)
    self.l2 = nn.Linear(input_size, 1, True)

  def forward(self, x: torch.Tensor):
    x = self.l1(x)
    x = self.l2(x)
    return x

In [6]:
def train_model(loader: torch.utils.data.DataLoader, model: nn.Module):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model.to(device)
    optimiser = torch.optim.SGD(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()  

    epoch_losses = []
    for i in range(300):
        epoch_loss = 0
        model.train()
        for _, (x, y) in enumerate(loader):
            # x, y = x.to(device), y.to(device)

            optimiser.zero_grad()
            y_pred = model(x)
            y = y.view(-1, 1).long()   
            loss = loss_fn(y_pred, y.float())
            epoch_loss += loss.item()
            loss.backward()
            optimiser.step()

        epoch_loss = epoch_loss / len(loader)
        epoch_losses.append(epoch_loss)
        print("Epoch: {}, Loss: {}".format(i, epoch_loss))
        

    return model, epoch_losses

In [7]:
model = Model(100)
train_model(train_loader, model)

Epoch: 0, Loss: 24.509523300024178
Epoch: 1, Loss: 2.611325447375958
Epoch: 2, Loss: 0.4755720289853903
Epoch: 3, Loss: 0.43251289083407474
Epoch: 4, Loss: 0.4521123766899109
Epoch: 5, Loss: 0.37471545201081496
Epoch: 6, Loss: 0.44176454498217654
Epoch: 7, Loss: 0.38695438320820147
Epoch: 8, Loss: 0.39326858062010545
Epoch: 9, Loss: 0.39264396062264073
Epoch: 10, Loss: 0.3937067481187674
Epoch: 11, Loss: 0.3913080463042626
Epoch: 12, Loss: 0.31762741849972653
Epoch: 13, Loss: 0.2972530470444606
Epoch: 14, Loss: 0.4351063599953285
Epoch: 15, Loss: 0.2809518781992105
Epoch: 16, Loss: 0.4787548780441284
Epoch: 17, Loss: 0.3974417791916774
Epoch: 18, Loss: 0.28825093232668364
Epoch: 19, Loss: 0.2703788624360011
Epoch: 20, Loss: 0.39315657432262713
Epoch: 21, Loss: 0.2666893142920274
Epoch: 22, Loss: 0.23404309382805458
Epoch: 23, Loss: 0.5518578439950943
Epoch: 24, Loss: 0.30162755113381606
Epoch: 25, Loss: 0.30102454125881195
Epoch: 26, Loss: 0.27570166610754454
Epoch: 27, Loss: 0.3901091

(Model(
   (l1): Linear(in_features=100, out_features=100, bias=True)
   (l2): Linear(in_features=100, out_features=1, bias=True)
 ),
 [24.509523300024178,
  2.611325447375958,
  0.4755720289853903,
  0.43251289083407474,
  0.4521123766899109,
  0.37471545201081496,
  0.44176454498217654,
  0.38695438320820147,
  0.39326858062010545,
  0.39264396062264073,
  0.3937067481187674,
  0.3913080463042626,
  0.31762741849972653,
  0.2972530470444606,
  0.4351063599953285,
  0.2809518781992105,
  0.4787548780441284,
  0.3974417791916774,
  0.28825093232668364,
  0.2703788624360011,
  0.39315657432262713,
  0.2666893142920274,
  0.23404309382805458,
  0.5518578439950943,
  0.30162755113381606,
  0.30102454125881195,
  0.27570166610754454,
  0.39010913784687334,
  0.24820623833399552,
  0.2533174088368049,
  0.45712697162077975,
  0.35251934024003834,
  0.23798309495815864,
  0.254391896036955,
  0.3128305799685992,
  0.37290606246544766,
  0.3215234245245273,
  0.31187910070786107,
  0.22288366

In [8]:
# do not remove – nothing to code here
# run this cell before moving on
# ensure get_accuracy from task 1.5 is defined

def get_accuracy(pred: torch.Tensor, label: torch.Tensor):
    y_pred = (pred > 0.5).long()
    label = label.long()

    return (y_pred == label).float().mean()

with torch.no_grad():
    model.eval()
    for i, data in enumerate(test_loader):
        x, y = data
        pred = model(x)
        y = y.view(-1, 1).long()
        print(pred.dtype, y.dtype)
        acc = get_accuracy(pred, y)
        print(f"Accuracy: {acc}")

torch.float32 torch.int64
Accuracy: 0.9587500095367432


In [9]:
torch.save(model.state_dict(), "ciphertext-model")